### Описание задания:
Решить задачу классификации при помощи обучения модели логистической регрессии. Целевая переменная — пол спортсмена. Качество модели оценивается путем подсчета метрик TPR, FPR и построения графиков ROC-кривой, Precision-Recall. 

1. Преобразуйте данные:
* проверьте наличие пропущенных значений. Преобразуйте/удалите пропуски по необходимости;
* закодируйте категориальные переменные числовыми значениями по необходимости.

In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('athletes.csv')
data.head()

In [ ]:
data.info()

In [ ]:
# Ключевые показатели для предсказания переменной sex - рост и вес
# Удалим пропуски, так как корректно по имеющимся признакам восстановить переменные height, weight сложно

data.dropna(subset=['height', 'weight'], inplace=True)

2. Разделите выборку на обучающее (80% данных) и тестовое (20% данных) подмножества.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['sex_encoder'] = le.fit_transform(data['sex'])
data.head()

In [ ]:
X = data[['height', 'weight']]
y = data['sex_encoder']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
predictions = model.predict_proba(X_test)[:, 1]
predictions

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

3. Постройте ROC-кривую с помощью функции roc_curve из библиотеки sklearn.metrics.

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, predictions)

In [ ]:
plt.plot(fpr, tpr, label='ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.show()

4. Вычислите значение ROC-AUC метрики с помощью функции roc_auc_score из библиотеки sklearn.metrics.

In [ ]:
from sklearn.metrics import auc

roc_auc = auc(fpr, tpr)
roc_auc

5. Реализуйте подсчет метрик TPR, FPR «вручную», без использования готовых функций из библиотеки sklearn.

In [ ]:
y_test

In [ ]:
predictions

In [ ]:
threshold = 0.5

predict_class = (predictions >= threshold).astype(int)
predict_class

In [ ]:
# Рассчитаем TP, FP, FN, TN 
TP = np.sum((predict_class == 1) & (y_test == 1))
FP = np.sum((predict_class == 1) & (y_test == 0))
FN = np.sum((predict_class == 0) & (y_test == 1))
TN = np.sum((predict_class == 0) & (y_test == 0))

# Теперь TPR и FPR
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

In [ ]:
print(f"TPR: {TPR:.3f}, FPR: {FPR}")

6. Постройте ROC-кривую с помощью вычисленных в п. 5 метрик. Объедините графики из п. 3 и п. 6 в один. Сравните, сделайте вывод.

In [ ]:
thresholds = np.linspace(0, 1, 100)

tpr = []
fpr = []

for threshold in thresholds:
    predict_class = (predictions >= threshold).astype(int)
    TP = np.sum((predict_class == 1) & (y_test == 1))
    FP = np.sum((predict_class == 1) & (y_test == 0))
    FN = np.sum((predict_class == 0) & (y_test == 1))
    TN = np.sum((predict_class == 0) & (y_test == 0))
    TPR = TP / (TP + FN)
    FPR = FP / (FP + TN)
    tpr.append(TPR)
    fpr.append(FPR)

In [ ]:
plt.plot(fpr, tpr, label='ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.show()

7. Постройте график Precision-Recall, используя метрики, посчитанные в п. 5.

In [ ]:
thresholds = np.linspace(0, 0.99, 1000)

precision_list = []
recall_list = []

for threshold in thresholds:
    predict_class = (predictions >= threshold).astype(int)
    TP = np.sum((predict_class == 1) & (y_test == 1))
    FP = np.sum((predict_class == 1) & (y_test == 0))
    FN = np.sum((predict_class == 0) & (y_test == 1))
    TN = np.sum((predict_class == 0) & (y_test == 0))
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall_TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
    precision_list.append(precision)
    recall_list.append(recall_TPR)

# Кривая начинается с точки (0,1)
precision_list.append(1)
recall_list.append(0)

In [ ]:
precision_list[-5:]

In [ ]:
recall_list[-5:]

In [ ]:
plt.plot(recall_list, precision_list, label='PR-curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR-curve')
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

precision, recall, thresholds = precision_recall_curve(y_test, predictions)

plt.plot(recall, precision, label='PR-curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR-curve')
plt.show()

8. Вычислите значение ROC-AUC метрики, используя метрики, посчитанные в п. 5.

In [ ]:
roc_auc = auc(fpr, tpr)
print(f"ROC-AUC: {roc_auc:.3f}")

9. Сформулируйте выводы по проделанной работе:
* как по полученным графикам сделать вывод о качестве модели? Как вы оцениваете обученную модель исходя из подсчитанных метрик?
* может ли ROC-кривая проходить ниже диагонали?

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

In [ ]:
print(f"Матрица ошибок:\n{cm}")

In [ ]:
TP, FP, TN, FN

* Согласно матрице ошибок модель хорошо распознает положительный класс: TP — 949, и отрицательный: TN — 738.
* Общая точность также показывает неплохой результат - 77%.
* В данном примере ROC-кривая не может быть ниже диагонали. При ROC-AUC = 50% полученная модель не лучше случайного угадывания. В нашем случае значение ROC-AUC = 85%. График кривой ROC и значение ROC-AUC показывают эффективность модели.